In [13]:
import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, AutoConfig
import os
from datasets import load_from_disk
os.environ['TRANSFORMERS_CACHE'] = '../transformers_cache'



In [25]:
data_test = load_from_disk("../datasets/gsm8k/val")
data_test


Dataset({
    features: ['question', 'answer'],
    num_rows: 1495
})

In [15]:
# data_train = load_from_disk("../datasets/gsm8k/train")
# data_train

In [16]:
import pandas as pd

In [69]:
path = '../outputs/gsm8k/LLaMA1B/generated_outputs_val_hyptune_0.3_0.6.json'
df = pd.read_json(path)

In [70]:
df.head()

,input,output,question,answer,model_answer,score
0,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistant.user\n\n...,"Janet, a third grade teacher, is picking up th...",\n\n35 + 5 + 1 = <<35+5+1=41>>41\n41 * 7 = <<4...,287,1
1,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistant.user\n\n...,"To make room for a new inventory of books, Gor...",\n\n25*0.3 = $7.50\n18*0.2 = $3.60\n21*0.2 = $...,26.30,1
2,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistant.user\n\n...,A young girl pours 23 blue beads and 16 yellow...,\n\n23 + 16 = 39\n39 / 3 = 13\n39 - 13 = 26\n2...,13,1
3,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistant.user\n\n...,Steve has 25 socks. He has 4 pairs of socks th...,\n\n25 - 4 = 21\n21 / 2 = 10.5\nSince you can'...,"46\n\nNow, solve the below question following...",0
4,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistant.user\n\n...,Theon's ship can move 15 nautical miles per ho...,\n\nTheon's speed = 15 nautical miles per hour...,6,1


In [71]:
df.shape

(1495, 6)

In [72]:
def get_answer_dataset(row):
    return row['answer'].split('####')[-1]
def get_answer_model(row):
    return row['output'].split('####')[-1]

In [73]:
df.drop(columns=['score','model_answer'],inplace=True)
df['model_answer']=df.apply(get_answer_model,axis=1)

In [74]:
df.head()

,input,output,question,answer,model_answer
0,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistant.user\n\n...,"Janet, a third grade teacher, is picking up th...",\n\n35 + 5 + 1 = <<35+5+1=41>>41\n41 * 7 = <<4...,287
1,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistant.user\n\n...,"To make room for a new inventory of books, Gor...",\n\n25*0.3 = $7.50\n18*0.2 = $3.60\n21*0.2 = $...,26.30
2,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistant.user\n\n...,A young girl pours 23 blue beads and 16 yellow...,\n\n23 + 16 = 39\n39 / 3 = 13\n39 - 13 = 26\n2...,13
3,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistant.user\n\n...,Steve has 25 socks. He has 4 pairs of socks th...,\n\n25 - 4 = 21\n21 / 2 = 10.5\nSince you can'...,"46\n\nNow, solve the below question following..."
4,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistant.user\n\n...,Theon's ship can move 15 nautical miles per ho...,\n\nTheon's speed = 15 nautical miles per hour...,6


In [75]:
# from evaluate import load
# from tqdm import tqdm
# exact_match_metric = load("exact_match")

In [76]:
# results = exact_match_metric.compute(predictions=[df.iloc[3]['model_answer']], references=[data_feedback["answer"][3].split("####")[-1]])

In [77]:
test = pd.DataFrame(data_test)
test.head()

,question,answer
0,"Janet, a third grade teacher, is picking up th...",Janet needs 35 lunches for the kids + 5 for th...
1,"To make room for a new inventory of books, Gor...",He gets 30% off any book over $22 and he buys ...
2,A young girl pours 23 blue beads and 16 yellow...,There are 23+16=<<23+16=39>>39 beads in the bo...
3,Steve has 25 socks. He has 4 pairs of socks th...,In each pair of socks there are 2/1 = <<2/1=2>...
4,Theon's ship can move 15 nautical miles per ho...,Theon's estimated time of arrival is 90/15 = <...


In [78]:
test['GT_Answer']=test.apply(get_answer_dataset,axis=1)

In [79]:
test.head()

,question,answer,GT_Answer
0,"Janet, a third grade teacher, is picking up th...",Janet needs 35 lunches for the kids + 5 for th...,308
1,"To make room for a new inventory of books, Gor...",He gets 30% off any book over $22 and he buys ...,95
2,A young girl pours 23 blue beads and 16 yellow...,There are 23+16=<<23+16=39>>39 beads in the bo...,10
3,Steve has 25 socks. He has 4 pairs of socks th...,In each pair of socks there are 2/1 = <<2/1=2>...,17
4,Theon's ship can move 15 nautical miles per ho...,Theon's estimated time of arrival is 90/15 = <...,3


In [80]:
# for i in tqdm(range(df.shape[0])):
#     df.loc[i,'score']=exact_match_metric.compute(predictions=[df.iloc[i]['model_answer']], references=[test.iloc[i]["GT_Answer"]])['exact_match']

In [10]:
# df.head()

In [82]:
df['score'] = (df['model_answer'] == test['GT_Answer']).astype(int)

In [11]:
# df.head()

In [84]:
# score= exact_match_metric.compute(predictions=df['model_answer'].tolist(), references=test["GT_Answer"].tolist())

In [36]:
# score

In [74]:
# df['score'] = (df['model_answer'] == feedback['GT_Answer']).astype(int)

In [75]:
# from tqdm import tqdm
# for i in tqdm(range(10)):
    

In [12]:
# df.head(10)

In [85]:
# df.drop(columns=['model_answer'],inplace=True)
df['GT_Answer']=test['GT_Answer']

In [13]:
# df.head()

In [87]:
# df.to_json("/scratch/workspace/wenlongzhao_umass_edu-analyze//Bidirectional-Teacher-Student-Communication-for-LLM-Alignment/outputs/gsm8k/LLaMA8B/generated_outputs1.json", orient='records', lines=True)

In [88]:
records = df.to_dict(orient='records')

In [89]:
import json
formatted_json = json.dumps(records, indent=4)


In [90]:
with open(path, "w") as f:
    f.write(formatted_json)

In [91]:
df['score'].sum()

407

In [92]:
df['score'].sum()/1495

0.27224080267558526

In [93]:
df.shape[0]

1495

In [6]:
import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, AutoConfig
import os
from datasets import load_from_disk
import pandas as pd
import json
# os.environ['TRANSFORMERS_CACHE'] = '../transformers_cache'



In [20]:
def get_answer_dataset(row):
    return row['answer'].split('####')[-1]
def get_answer_model(row):
    return row['output'].split('####')[-1]
def get_score(data_path, output_path):
    '''
        data_path = Path to HF dataset which has GT
        output_path = path to json file containing model outputs
    '''
    test = pd.DataFrame(load_from_disk(data_path))    
    df = pd.read_json(output_path)
    # df.drop(columns=['score','model_answer'],inplace=True)
    df['model_answer'] = df.apply(get_answer_model,axis=1)
    test['GT_Answer'] = test.apply(get_answer_dataset, axis=1)
    df['GT_Answer']=test['GT_Answer']
    # print(f'DF Shape: {df.shape}, Test Shape: {test.shape}')
    # exact_match_metric = load("exact_match")
    
    df['score'] = (df['model_answer'] == test['GT_Answer']).astype(int)
    
    # # for i in tqdm(range(df.shape[0])):
    # for i in range(df.shape[0]):
    #     df.loc[i,'score'] = exact_match_metric.compute(predictions = [df.iloc[i]['model_answer']], references = [test.iloc[i]['GT_Answer']])['exact_match']
    
    # df.drop(columns=['model_answer'], inplace=True)
    score = df['score'].sum()/df.shape[0]
    # Write back the exact scores to output_path
    records = df.to_dict(orient = 'records')
    formatted_json = json.dumps(records, indent=4)
    
    with open(output_path,'w') as f:
        f.write(formatted_json)
    
    return score, df['score'].sum()

In [22]:
top_p = [0.95,0.9,0.85,0.8,0.75,0.7,0.65,0.6]
temp = [0.3,0.5,0.7,0.9,1.1,1.3,1.5,1.7,1.9]
# top_p = [0.6]
# temp = [0.3]
data_eval_path = "../datasets/gsm8k/val/"
model_name = "meta-llama/Llama-3.2-3B-Instruct"
exp_runs=[]
for t in temp:
    for p in top_p:
        output_file_name = f"../outputs/gsm8k/LLaMA3B/generated_outputs_val_hyptune_{t}_{p}.json" 
        score, s = get_score(data_eval_path,output_file_name)
        exp_runs.append({'model':model_name,'temp':t,'top_p':p,'score':score})
        print(f"\n\nModel Name: {model_name}\tTemp: {t}\tTop_P: {p}\tScore: {score}\tSum: {s}")
output_file_name = f"../outputs/gsm8k/hyperparameter3B.json" 
with open(output_file_name, "w") as f:
    json.dump(exp_runs, f, indent=4)  



Model Name: meta-llama/Llama-3.2-3B-Instruct	Temp: 0.3	Top_P: 0.95	Score: 0.7946488294314381	Sum: 1188


Model Name: meta-llama/Llama-3.2-3B-Instruct	Temp: 0.3	Top_P: 0.9	Score: 0.8	Sum: 1196


Model Name: meta-llama/Llama-3.2-3B-Instruct	Temp: 0.3	Top_P: 0.85	Score: 0.7819397993311037	Sum: 1169


Model Name: meta-llama/Llama-3.2-3B-Instruct	Temp: 0.3	Top_P: 0.8	Score: 0.7819397993311037	Sum: 1169


Model Name: meta-llama/Llama-3.2-3B-Instruct	Temp: 0.3	Top_P: 0.75	Score: 0.7852842809364549	Sum: 1174


Model Name: meta-llama/Llama-3.2-3B-Instruct	Temp: 0.3	Top_P: 0.7	Score: 0.7939799331103679	Sum: 1187


Model Name: meta-llama/Llama-3.2-3B-Instruct	Temp: 0.3	Top_P: 0.65	Score: 0.7953177257525084	Sum: 1189


Model Name: meta-llama/Llama-3.2-3B-Instruct	Temp: 0.3	Top_P: 0.6	Score: 0.7839464882943143	Sum: 1172


Model Name: meta-llama/Llama-3.2-3B-Instruct	Temp: 0.5	Top_P: 0.95	Score: 0.782608695652174	Sum: 1170


Model Name: meta-llama/Llama-3.2-3B-Instruct	Temp: 0.5	Top_P: 0.9	Score: 

In [17]:
top_p = [0.95,0.9,0.85,0.8,0.75,0.7,0.65,0.6]
temp = [0.3,0.5,0.7,0.9,1.1,1.3,1.5,1.7,1.9]
for t in temp:
    for p in top_p:
        output_file_name = f"../outputs/gsm8k/LLaMA1B/generated_outputs_val_hyptune_{t}_{p}.json" 
        df = pd.read_json(output_file_name)
        print(output_file_name)
        df.drop(columns=['score','model_answer'],inplace=True)
        records = df.to_dict(orient = 'records')
        formatted_json = json.dumps(records, indent=4)

        with open(output_file_name,'w') as f:
            f.write(formatted_json)

../outputs/gsm8k/LLaMA1B/generated_outputs_val_hyptune_0.3_0.95.json
../outputs/gsm8k/LLaMA1B/generated_outputs_val_hyptune_0.3_0.9.json
../outputs/gsm8k/LLaMA1B/generated_outputs_val_hyptune_0.3_0.85.json
../outputs/gsm8k/LLaMA1B/generated_outputs_val_hyptune_0.3_0.8.json
../outputs/gsm8k/LLaMA1B/generated_outputs_val_hyptune_0.3_0.75.json
../outputs/gsm8k/LLaMA1B/generated_outputs_val_hyptune_0.3_0.7.json
../outputs/gsm8k/LLaMA1B/generated_outputs_val_hyptune_0.3_0.65.json
../outputs/gsm8k/LLaMA1B/generated_outputs_val_hyptune_0.3_0.6.json
../outputs/gsm8k/LLaMA1B/generated_outputs_val_hyptune_0.5_0.95.json
../outputs/gsm8k/LLaMA1B/generated_outputs_val_hyptune_0.5_0.9.json
../outputs/gsm8k/LLaMA1B/generated_outputs_val_hyptune_0.5_0.85.json
../outputs/gsm8k/LLaMA1B/generated_outputs_val_hyptune_0.5_0.8.json
../outputs/gsm8k/LLaMA1B/generated_outputs_val_hyptune_0.5_0.75.json
../outputs/gsm8k/LLaMA1B/generated_outputs_val_hyptune_0.5_0.7.json
../outputs/gsm8k/LLaMA1B/generated_output